In [1]:
import pandas as pd
import numpy as np
import os
import sys
import time
from PIL import ImageOps, Image
import h5py
from tqdm import tqdm

F = os.getenv("SLURM_TMPDIR")
f = h5py.File(os.path.join(F,"ucf101_train.hdf5"), "w")

def pil_loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')    
        
split = '../data/ucf101/train_split%02d.csv' % 1
video_info_train = pd.read_csv(split, header=None)

split = '../data/ucf101/test_split%02d.csv' % 1 
video_info_val = pd.read_csv(split, header=None)


# filter out too short videos:
drop_idx = []
for idx, row in video_info_train.iterrows():
    vpath, vlen = row
    if vlen-8*5*3 <= 0:
        drop_idx.append(idx)
video_info_train = video_info_train.drop(drop_idx, axis=0)

drop_idx = []
for idx, row in video_info_val.iterrows():
    vpath, vlen = row
    if vlen-8*5*3 <= 0:
        drop_idx.append(idx)
video_info_val = video_info_val.drop(drop_idx, axis=0)

num_video_train = len(video_info_train)
num_video_val = len(video_info_val)

print(num_video_train,num_video_val)

vpath = list()

for idx in range(10):#tqdm(range(num_video_train),total=num_video_train):
    grp = f.create_group(str(idx))
    vpath, vlen = video_info_train.iloc[idx]
    seq = pil_loader(os.path.join(vpath, 'image_%05d.jpg' % (1)))
    (w,h) = seq.size
    seq = [pil_loader(os.path.join(vpath, 'image_%05d.jpg' % (i+1))) for i in range(vlen)]
    M = np.stack(seq,axis=0)
    dset = f[str(idx)].create_dataset('video', (vlen,h,w,3),data=M,dtype='uint8',chunks = True,
                                      compression="gzip",compression_opts=9)
    del M
    
    
f.close()




6809 2647


FileNotFoundError: [Errno 2] No such file or directory: '/network/tmp1/bakhtias/UCF101_full/UCF101_full/frame/ApplyEyeMakeup/v_ApplyEyeMakeup_g08_c03/image_00001.jpg'

In [ ]:
f = h5py.File(os.path.join(F,"ucf101_val.hdf5"), "w")

for idx in tqdm(range(num_video_val),total=num_video_val):
    grp = f.create_group(str(idx))
    vpath, vlen = video_info_val.iloc[idx]
    seq = pil_loader(os.path.join(vpath, 'image_%05d.jpg' % (1)))
    (w,h) = seq.size
    seq = [pil_loader(os.path.join(vpath, 'image_%05d.jpg' % (i+1))) for i in range(vlen)]
    M = np.stack(seq,axis=0)
    dset = f[str(idx)].create_dataset('video', (vlen,h,w,3),data=M,dtype='uint8',compression="gzip")

f.close()


In [ ]:
test = f['9'].require_dataset('video',shape=(261,240,320,3),dtype='uint8')

In [ ]:

# import matplotlib.pyplot as plt
test = dset[0,:,:,:]
plt.imshow(test)
plt.show()